# Python Script for MODIS Data Processing: NDVI and NIRv Calculation

## Description:
This script processes MODIS surface reflectance data to calculate the **NDVI (Normalized Difference Vegetation Index)** and **NIRv (Near-Infrared Vegetation Index)** for the period **2000–2024** using the **MODIS 09A1 collection** (surface reflectance). The main tasks include:

- Downloading MODIS data within a specified bounding box and date range using the STAC catalog.
- Computing NDVI and NIRv using the MODIS bands (`sur_refl_b01` and `sur_refl_b02`), while applying a quality control mask.
- Aggregating the data to monthly intervals and calculating the mean for NDVI and NIRv.
- Saving the results as **GeoTIFF** files (NDVI and NIRv) and a **NetCDF** file for further analysis.

## Workflow:
1. **Bounding Box and Date Range**: Define the area of interest and time period.
2. **STAC Catalog Search**: Query the STAC catalog for MODIS data based on the bounding box and date range.
3. **Data Loading**: Load the MODIS data using efficient chunking to handle large datasets.
4. **Quality Control Mask**: Apply a QC mask to filter out invalid pixels.
5. **NDVI and NIRv Calculation**: Calculate NDVI and NIRv using surface reflectance bands.
6. **Monthly Aggregation**: Resample the data to monthly intervals and compute the mean.
7. **Saving Outputs**: Save NDVI and NIRv as GeoTIFF files and the aggregated dataset as a NetCDF file.

## Data Requirements:
- **MODIS 09A1 Collection** (surface reflectance bands: `sur_refl_b01`, `sur_refl_b02`, and `sur_refl_qc_500m`).
- Access to the **Planetary Computer STAC catalog** for data search and retrieval.

## Outputs:
- **GeoTIFF** files for monthly means of NDVI and NIRv.
- **NetCDF** file containing the aggregated NDVI and NIRv data over the period 2000–2024.


In [1]:
import dask.distributed
from pystac_client import Client
import planetary_computer
import xarray as xr
import odc.stac
import rioxarray

# Initialize Dask distributed client
client = dask.distributed.Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 16,Total memory: 23.70 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:62201,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 16
Started: Just now,Total memory: 23.70 GiB
Comm: tcp://127.0.0.1:62231,Total threads: 4
Dashboard: http://127.0.0.1:62235/status,Memory: 5.93 GiB
Nanny: tcp://127.0.0.1:62204,


In [ ]:
# 1. Define Bounding Box and Parameters
# BBOX specifies the region of interest (ROI) for the data search (min_lon, min_lat, max_lon, max_lat).
# MODIS_COLLECTION is the specific collection from which data is fetched.
# DATETIME defines the period for which the MODIS data is requested.
BBOX = [5.8663153, 47.2701114, 15.0419319, 55.058347]
MODIS_COLLECTION = "modis-09A1-061"
DATETIME = ("2000-01-01", "2024-12-31")

# 2. Load the STAC catalog and search items
# Using Planetary Computer's STAC API, we search for MODIS data within the specified bounding box and datetime range.
catalog = Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")
item_search = catalog.search(
    collections=[MODIS_COLLECTION], bbox=BBOX, datetime=DATETIME
)
# Sign the item collection to authenticate and get the data
item_collection = planetary_computer.sign_item_collection(item_search.item_collection())

# 3. Load data with optimal chunking for memory efficiency
# We load the requested data using `odc.stac.load()`, specifying which bands to retrieve 
# and the chunking strategy to handle large datasets efficiently.
ds = odc.stac.load(
    item_collection,
    bands=["sur_refl_b01", "sur_refl_b02", "sur_refl_qc_500m"],
    chunks={'time': 50, 'x': 1000, 'y': 1000},
    bbox=BBOX,
    crs="epsg:31467",  
    resolution=1000,
)

# 4. Define QC mask functions
# `decode_qc` function decodes the quality control bits to assess the quality of each pixel.
def decode_qc(qc_array, bit_position, bit_length):
    return (qc_array >> bit_position) & ((1 << bit_length) - 1)

# `create_quality_mask` function creates a mask based on the pixel quality (only valid pixels are kept).
def create_quality_mask(qc_band):
    pixel_quality = decode_qc(qc_band, 0, 2)
    return pixel_quality == 0

# Precompute the quality mask using the QC band from the dataset
qc_mask = create_quality_mask(ds["sur_refl_qc_500m"])

# 5. Apply mask and calculate NDVI and NIRv in one step
# `compute_ndvi_nirv` function computes the NDVI and NIRv using the relevant bands 
# (sur_refl_b01 for blue and sur_refl_b02 for red).
def compute_ndvi_nirv(b01, b02, mask):
    b01_masked = b01.where(mask)
    b02_masked = b02.where(mask)
    b01_masked = b01_masked * 0.0001  # Scaling factor for the reflectance data
    b02_masked = b02_masked * 0.0001
    ndvi = (b02_masked - b01_masked) / (b02_masked + b01_masked)  # NDVI formula
    nirv = ndvi * b02_masked  # NIRv formula
    return ndvi, nirv

# Compute NDVI and NIRv for the dataset
ndvi, nirv = compute_ndvi_nirv(ds['sur_refl_b01'], ds['sur_refl_b02'], qc_mask)

# 6. Add the calculated NDVI and NIRv to the dataset
ds['NDVI'] = ndvi
ds['NIRv'] = nirv

# 7. Persist the dataset to optimize further operations and avoid recomputing
ds = ds.persist()

# 8. Resample and compute monthly means
# Resample the data by month (time='ME') and compute the mean for each month.
ds_monthly = ds[['NDVI', 'NIRv']].resample(time='ME').mean()

# 9. Ensure spatial information is set for the dataset
# Set the CRS for the dataset to EPSG:31467.
ds_monthly['NDVI'].rio.write_crs("epsg:31467", inplace=True)
ds_monthly['NIRv'].rio.write_crs("epsg:31467", inplace=True)

# 10. Write the NDVI and NIRv data to GeoTIFF files
# Save NDVI as a GeoTIFF file.
ndvi_tif = "NDVI_monthly_means.tif"
ds_monthly['NDVI'].rio.to_raster(ndvi_tif)

# Save NIRv as a GeoTIFF file.
nirv_tif = "NIRv_monthly_means.tif"
ds_monthly['NIRv'].rio.to_raster(nirv_tif)

print(f"NDVI saved to {ndvi_tif}")
print(f"NIRv saved to {nirv_tif}")

# 11. Save the aggregated dataset to a NetCDF file
ds_monthly.to_netcdf("aggregated_monthly_ndvi_nirv_optimized.nc")